In [1]:
ROOT = "../../"
import sys
sys.path.append(ROOT)
#
from src.constants import (
    ML_MOVIES_PATH, ML_RATINGS_PATH, ML_USERS_PATH,
    BC_BOOKS_PATH, BC_RATINGS_PATH, BC_USERS_PATH
)
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Data

## MovieLens

### Users

In [2]:
# 1) Read lines and split them by "::"
# 2) Select the columns and cast them to the correct types
ml_ocupations = [
    "other",
	"academic/educator",
	"artist",
	"clerical/admin",
	"college/grad student",
	"customer service",
	"doctor/health care",
	"executive/managerial",
	"farmer",
	"homemaker",
	"K-12 student",
	"lawyer",
	"programmer",
	"retired",
	"sales/marketing",
	"scientist",
	"self-employed",
	"technician/engineer",
	"tradesman/craftsman",
	"unemployed",
	"writer",
]
ml_occupation_map = {i: occupation for i, occupation in enumerate(ml_ocupations)}
ml_users_df = pl.scan_csv(ROOT + ML_USERS_PATH, has_header=False, truncate_ragged_lines=True, encoding="utf8-lossy").select([
    pl.col("column_1").str.split("::")
]).select([
    pl.col("column_1").list.get(0).alias("UserID").cast(pl.Int32),
    pl.col("column_1").list.get(1).alias("Gender"),
    pl.col("column_1").list.get(2).alias("Age").cast(pl.Int32),
    pl.col("column_1").list.get(3).alias("Occupation").cast(pl.Int32).map_dict(ml_occupation_map),
    pl.col("column_1").list.get(4).alias("Zip-code"),
])

ml_users_df.head().collect()

/var/folders/16/2kmt14y56rqd2xkgkp53tr080000gn/T/ipykernel_47037/792700600.py:33: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  pl.col("column_1").list.get(3).alias("Occupation").cast(pl.Int32).map_dict(ml_occupation_map),


UserID,Gender,Age,Occupation,Zip-code
i32,str,i32,str,str
1,"""F""",1,"""K-12 student""","""48067"""
2,"""M""",56,"""self-employed""","""70072"""
3,"""M""",25,"""scientist""","""55117"""
4,"""M""",45,"""executive/managerial""","""02460"""
5,"""M""",25,"""writer""","""55455"""


### Movies


In [3]:
# 1) Read lines and split them by "::"
# 2) Select the columns and cast them to the correct types
# 3) Extract the year from the title column
# 4) If year is not null, remove it from the title
#   If the title is "Toy Story (1995)", the title should be "Toy Story" and the year should be 1995. 
#   If the title does not have a year, the year should be null.
ml_movies_df = pl.scan_csv(ROOT + ML_MOVIES_PATH, has_header=False, truncate_ragged_lines=True, encoding="utf8-lossy").select([
    pl.col("column_1").str.split("::")
]).select([
    pl.col("column_1").list.get(0).alias("MovieID").cast(pl.Int32),
    pl.col("column_1").list.get(1).alias("Title"),
    pl.col("column_1").list.get(2).str.split("|").alias("Genres")
]).with_columns([
    pl.col("Title").str.extract(r"\((\d{4})\)$").alias("Year").cast(pl.Int32),
]).with_columns([
    # If Year is not null, remove it from the title
    pl.when(pl.col("Year").is_null()).then(pl.col("Title")).otherwise(
        pl.col("Title").str.slice(0, pl.col("Title").str.find(" \((\d{4})\)$"))
    ).alias("Title")
])
# 4) Get possible genres
ml_genres = ml_movies_df.select(pl.col("Genres").explode().unique()).collect()
# 5) Make dummy variable for each genre
ml_movies_df = ml_movies_df.with_columns([
    pl.col("Genres").list.contains(genre[0]).alias(f"Is{genre[0]}")
    for genre in ml_genres.rows() if genre[0] is not None
])

In [4]:
ml_movies_df.collect()

MovieID,Title,Genres,Year,IsThriller,IsCrime,IsAdventure,IsAnimation,IsWar,IsNone,IsFantasy,IsMystery,IsHorror,IsDrama,IsDocumentary,IsChildren's,IsAction,IsWestern,IsComedy,IsSci-Fi,IsRomance,IsMusical,IsFilm-Noir
i32,str,list[str],i32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
1,"""Toy Story""","[""Animation"", ""Children's"", ""Comedy""]",1995,false,false,false,true,false,null,false,false,false,false,false,true,false,false,true,false,false,false,false
2,"""Jumanji""","[""Adventure"", ""Children's"", ""Fantasy""]",1995,false,false,true,false,false,null,true,false,false,false,false,true,false,false,false,false,false,false,false
3,"""Grumpier Old Men""","[""Comedy"", ""Romance""]",1995,false,false,false,false,false,null,false,false,false,false,false,false,false,false,true,false,true,false,false
4,"""Waiting to Exhale""","[""Comedy"", ""Drama""]",1995,false,false,false,false,false,null,false,false,false,true,false,false,false,false,true,false,false,false,false
5,"""Father of the Bride Part II""","[""Comedy""]",1995,false,false,false,false,false,null,false,false,false,false,false,false,false,false,true,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3948,"""Meet the Parents""","[""Comedy""]",2000,false,false,false,false,false,null,false,false,false,false,false,false,false,false,true,false,false,false,false
3949,"""Requiem for a Dream""","[""Drama""]",2000,false,false,false,false,false,null,false,false,false,true,false,false,false,false,false,false,false,false,false
3950,"""Tigerland""","[""Drama""]",2000,false,false,false,false,false,null,false,false,false,true,false,false,false,false,false,false,false,false,false


In [5]:
ml_movies_df.null_count().collect()

MovieID,Title,Genres,Year,IsThriller,IsCrime,IsAdventure,IsAnimation,IsWar,IsNone,IsFantasy,IsMystery,IsHorror,IsDrama,IsDocumentary,IsChildren's,IsAction,IsWestern,IsComedy,IsSci-Fi,IsRomance,IsMusical,IsFilm-Noir
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,993,993,0,0,0,0,0,3883,0,0,0,0,0,0,0,0,0,0,0,0,0


### Ratings

In [6]:
# 1) Read lines and split them by "::"
# 2) Select columns and cast them to the correct type
ml_ratings_df = pl.scan_csv(ROOT + ML_RATINGS_PATH, has_header=False).select([
    pl.col("column_1").str.split("::")
]).select([
    pl.col("column_1").list.get(0).alias("UserID").cast(pl.Int32),
    pl.col("column_1").list.get(1).alias("MovieID").cast(pl.Int32),
    pl.col("column_1").list.get(2).alias("Rating").cast(pl.Int32),
    pl.col("column_1").list.get(3).alias("Timestamp").cast(pl.Int32)
])

ml_ratings_df.collect()

UserID,MovieID,Rating,Timestamp
i32,i32,i32,i32
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
…,…,…,…
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [7]:
ml_ratings_df.null_count().collect()

UserID,MovieID,Rating,Timestamp
u32,u32,u32,u32
0,0,0,0


### Join

In [8]:
# Join dataframes
ml_df = ml_ratings_df.join(ml_movies_df, on="MovieID").join(ml_users_df, on="UserID")
ml_df.collect()

UserID,MovieID,Rating,Timestamp,Title,Genres,Year,IsThriller,IsCrime,IsAdventure,IsAnimation,IsWar,IsNone,IsFantasy,IsMystery,IsHorror,IsDrama,IsDocumentary,IsChildren's,IsAction,IsWestern,IsComedy,IsSci-Fi,IsRomance,IsMusical,IsFilm-Noir,Gender,Age,Occupation,Zip-code
i32,i32,i32,i32,str,list[str],i32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,i32,str,str
1,1193,5,978300760,"""One Flew Over the Cuckoo's Nes…","[""Drama""]",1975,false,false,false,false,false,null,false,false,false,true,false,false,false,false,false,false,false,false,false,"""F""",1,"""K-12 student""","""48067"""
1,661,3,978302109,"""James and the Giant Peach""","[""Animation"", ""Children's"", ""Musical""]",1996,false,false,false,true,false,null,false,false,false,false,false,true,false,false,false,false,false,true,false,"""F""",1,"""K-12 student""","""48067"""
1,914,3,978301968,"""My Fair Lady""","[""Musical"", ""Romance""]",1964,false,false,false,false,false,null,false,false,false,false,false,false,false,false,false,false,true,true,false,"""F""",1,"""K-12 student""","""48067"""
1,3408,4,978300275,"""Erin Brockovich""","[""Drama""]",2000,false,false,false,false,false,null,false,false,false,true,false,false,false,false,false,false,false,false,false,"""F""",1,"""K-12 student""","""48067"""
1,2355,5,978824291,"""Bug's Life""",null,null,false,false,false,false,false,null,false,false,false,false,false,false,false,false,false,false,false,false,false,"""F""",1,"""K-12 student""","""48067"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6040,1091,1,956716541,"""Weekend at Bernie's""","[""Comedy""]",1989,false,false,false,false,false,null,false,false,false,false,false,false,false,false,true,false,false,false,false,"""M""",25,"""doctor/health care""","""11106"""
6040,1094,5,956704887,"""Crying Game""",null,null,false,false,false,false,false,null,false,false,false,false,false,false,false,false,false,false,false,false,false,"""M""",25,"""doctor/health care""","""11106"""
6040,562,5,956704746,"""Welcome to the Dollhouse""","[""Comedy"", ""Drama""]",1995,false,false,false,false,false,null,false,false,false,true,false,false,false,false,true,false,false,false,false,"""M""",25,"""doctor/health care""","""11106"""


In [9]:
ml_df.null_count().collect()

UserID,MovieID,Rating,Timestamp,Title,Genres,Year,IsThriller,IsCrime,IsAdventure,IsAnimation,IsWar,IsNone,IsFantasy,IsMystery,IsHorror,IsDrama,IsDocumentary,IsChildren's,IsAction,IsWestern,IsComedy,IsSci-Fi,IsRomance,IsMusical,IsFilm-Noir,Gender,Age,Occupation,Zip-code
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,243234,243234,0,0,0,0,0,1000209,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Book-Crossing

In [10]:
bc_books_df = pl.scan_csv(ROOT + BC_BOOKS_PATH, separator=";")
bc_books_df.collect()

ISBN,Title,Author,Year,Publisher
str,str,str,i64,str
"""0195153448""","""Classical Mythology""","""Mark P. O. Morford""",2002,"""Oxford University Press"""
"""0002005018""","""Clara Callan""","""Richard Bruce Wright""",2001,"""HarperFlamingo Canada"""
"""0060973129""","""Decision in Normandy""","""Carlo D'Este""",1991,"""HarperPerennial"""
"""0374157065""","""Flu: The Story of the Great In…","""Gina Bari Kolata""",1999,"""Farrar Straus Giroux"""
"""0393045218""","""The Mummies of Urumchi""","""E. J. W. Barber""",1999,"""W. W. Norton & Company"""
…,…,…,…,…
"""0440400988""","""There's a Bat in Bunk Five""","""Paula Danziger""",1988,"""Random House Childrens Pub (Mm…"
"""0525447644""","""From One to One Hundred""","""Teri Sloat""",1991,"""Dutton Books"""
"""006008667X""","""Lily Dale : The True Story of …","""Christine Wicker""",2004,"""HarperSanFrancisco"""


In [11]:
bc_books_df.null_count().collect()

ISBN,Title,Author,Year,Publisher
u32,u32,u32,u32,u32
0,0,1,0,2


In [12]:
bc_users_df = pl.scan_csv(ROOT + BC_USERS_PATH, separator=";").filter(
    pl.col("Age").is_between(5, 100)
)
bc_users_df.collect()

User-ID,Age
i64,i64
2,18
4,17
6,61
10,26
11,14
…,…
278849,23
278851,33
278852,32


In [13]:
bc_users_df.null_count().collect()

User-ID,Age
u32,u32
0,0


In [16]:
bc_ratings_df = pl.scan_csv(ROOT + BC_RATINGS_PATH, separator=";").filter(
    pl.col("User-ID").is_in(bc_users_df.select("User-ID").collect()),
)
bc_ratings_df.collect()

User-ID,ISBN,Rating
i64,str,i64
276727,"""0446520802""",0
276729,"""052165615X""",3
276729,"""0521795028""",6
276733,"""2080674722""",0
276737,"""0600570967""",6
…,…,…
276697,"""8445072897""",0
276706,"""0679447156""",0
276709,"""0515107662""",10


In [17]:
bc_ratings_df.null_count().collect()

User-ID,ISBN,Rating
u32,u32,u32
0,0,0
